In [1]:
import numpy as np
import pandas as pd
import seaborn as sns

from matplotlib import pyplot as plt
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import PowerTransformer
from feature_engine.outliers import Winsorizer

In [6]:
tb = pd.read_csv('../data/../data/train.csv', index_col='id')
tb_data = tb.sample(n=50000, random_state=16)
tb_s1 = tb.sample(n=50000, random_state=32)
tb_s2 = tb.sample(n=50000, random_state=64)

In [3]:
feature_eng = Pipeline(steps=[('outliers', Winsorizer(capping_method='iqr',
                                                      tail='both')),
                              ('power', PowerTransformer()),
                              ('range', MinMaxScaler())])
feature_eng.get_params()

{'memory': None,
 'steps': [('outliers', Winsorizer(capping_method='iqr', tail='both')),
  ('power', PowerTransformer()),
  ('range', MinMaxScaler())],
 'verbose': False,
 'outliers': Winsorizer(capping_method='iqr', tail='both'),
 'power': PowerTransformer(),
 'range': MinMaxScaler(),
 'outliers__capping_method': 'iqr',
 'outliers__fold': 3,
 'outliers__missing_values': 'raise',
 'outliers__tail': 'both',
 'outliers__variables': None,
 'power__copy': True,
 'power__method': 'yeo-johnson',
 'power__standardize': True,
 'range__clip': False,
 'range__copy': True,
 'range__feature_range': (0, 1)}

In [7]:
tb_data_fe = pd.DataFrame(feature_eng.fit_transform(tb_data.loc[:, :'f99']), columns=tb_data.columns[:100])
tb_s1_fe = pd.DataFrame(feature_eng.fit_transform(tb_s1.loc[:, :'f99']), columns=tb_data.columns[:100])
tb_s2_fe = pd.DataFrame(feature_eng.fit_transform(tb_s2.loc[:, :'f99']), columns=tb_data.columns[:100])

In [8]:
tb_s1_Y = tb_s1['target']
tb_s2_Y = tb_s2['target']